In [1]:
"""
Name: movement.ipynb
Authors: Stephan Meighen-Berger, Martina Karl
Example how to create a video of the simulation
Requires mpl_toolkits and imageio
"""

'\nName: movement.ipynb\nAuthors: Stephan Meighen-Berger, Martina Karl\nExample how to create a video of the simulation\nRequires mpl_toolkits and imageio\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import matplotlib.colors as colors
import imageio

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day as FD
from fourth_day import config

In [6]:
# Scenario
config['scenario']['population size'] = 500
config['scenario']['duration'] = 600
config['scenario']['organism movement'] = False
config['scenario']['exclusion'] = True
config['scenario']['injection']['rate'] = 8
config['scenario']['injection']['y range'] = [0., 10.]
config['scenario']['light prop'] = {
            "switch": True,
            "x_pos": 5.,
            "y_pos": 10.,
        }
config['scenario']['detector'] = {
    "switch": True,
    "type": "PMTSpec",
    "response": True,
    "acceptance": "Flat",
    "mean detection prob": 0.5
}
# Organisms 
config['organisms']['emission fraction'] = 0.1
config['organisms']['alpha'] = 1.
# Geometry
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    'offset': None,
}
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 26.,
    'y_length': 10.,
    "offset": np.array([0., 0.]),
}
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.15,
    "x_pos": 2.,
    "y_pos": 5.,
}
# Water
config['water']['model']['name'] = 'custom' # 'potential cylinder'
config['water']['model']['directory'] = "../data/current/long_run/"
config['water']['model']['time step'] = 1
config['advanced']['starting step'] = 0

In [7]:
statistics = []
wavelengths = []
measured = []
seeds = [10, 1337]
for seed in seeds:
    # General
    config["general"]["random state seed"] = seed
    # Creating a fourth_day object
    fd = FD()
    # Launching solver
    fd.sim()
    statistics.append(fd.statistics)
    wavelengths.append(fd.wavelengths)
    measured.append(fd.measured)

In [16]:
# Plotting standards
std_size = 20
fontsize = 12.
lw=3.
h_length=1.5

In [17]:
# Offset
off_set = 0

In [18]:
courseness = 1000
vmax = 0.01
vmin = 0.

In [19]:
# Random noise
noise = []
for i, _ in enumerate(fd.measured.keys()):
    noise.append(np.abs(np.random.normal(2e2, 1e2, len(fd.t))))
noise = np.array(noise)
x_grid = np.linspace(0., config['geometry']['volume']['x_length'], courseness)
y_grid = np.linspace(0., config['geometry']['volume']['y_length'], courseness)
X, Y = np.meshgrid(x_grid, y_grid)
all_points = []
for i in range(0, len(x_grid)):
    all_points.append(np.vstack((X[i], Y[i])).T)
for i, _ in tqdm(enumerate(fd.t[off_set:])):
    q = i + off_set
    figure, ((ax, ax3), (ax1, ax4)) = plt.subplots(2, 2, figsize=(std_size, std_size * 6. / 8.))
    # The water current
    x_vel = []
    y_vel = []
    grad = []
    for j in range(0, len(x_grid)):
        # x_vel_tmp, y_vel_tmp, vel_abs = fd._current.velocities(all_points[j], q + config['advanced']['starting step'])
        grad_tmp = fd._current.gradients(all_points[j], q + config['advanced']['starting step'])
        # x_vel.append(x_vel_tmp)
        # y_vel.append(y_vel_tmp)
        grad.append(grad_tmp)
    grad = np.array(grad)
    ax.imshow(grad.reshape(courseness, courseness), cmap='plasma',
              vmax=vmax, vmin=vmin, extent=[0., 26., 0., 10.])
    # ax.quiver(x_grid, y_grid, x_vel, y_vel, grad)
    # The emitters
    emission_mask = statistics[0][q].loc[:, 'is_emitting']
    ax.scatter(statistics[0][q].loc[~emission_mask, 'pos_x'].values,
               statistics[0][q].loc[~emission_mask, 'pos_y'].values,
               color='g',
               s=10.
    )
    ax.scatter(statistics[0][q].loc[emission_mask, 'pos_x'].values,
               statistics[0][q].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=10.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw/2, color='k')
    # Additional options
    ax.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax.title=ax.set_title('Population t = %.f s' %(fd.t[q]), fontsize=fontsize)
    ax.set_xlim(0., 10.)
    ax.set_ylim(3., 7.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, detector in enumerate(fd.measured.keys()):
        ax1.plot((fd.t)[:q+1],
             measured[0][detector].values[:q+1] + noise[j, :q+1],
             lw=lw, label=detector)
    #ax1.set_ylim(1e6, 1e11)
    ax1.set_xscale('linear')
    ax1.set_yscale('log')
    ax1.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax1.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax1.grid(True)
    h, l = ax1.get_legend_handles_labels()
    lgd1 = ax1.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.05),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax1.set_xlim(fd.t[off_set], max(fd.t))
    ax1.set_ylim(2e1,1e5)
    # ----------------------------------------------------------------------------------------------------
    # Water current
    ax3.imshow(grad.reshape(courseness, courseness), cmap='plasma',
               vmax=vmax, vmin=vmin, extent=[0., 26., 0., 10.])
    # Emitters
    emission_mask = statistics[1][q].loc[:, 'is_emitting']
    ax3.scatter(statistics[1][q].loc[~emission_mask, 'pos_x'].values,
               statistics[1][q].loc[~emission_mask, 'pos_y'].values,
               color='g',
               s=10.
    )
    ax3.scatter(statistics[1][q].loc[emission_mask, 'pos_x'].values,
               statistics[1][q].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=10.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax3.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw/2, color='k')
    # Additional options
    ax3.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax3.title=ax3.set_title('Population t = %.f s' %(fd.t[q]), fontsize=fontsize)
    ax3.set_xlim(0., 10.)
    ax3.set_ylim(3., 7.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, detector in enumerate(fd.measured.keys()):
        ax4.plot((fd.t)[:q+1],
             measured[1][detector].values[:q+1] + noise[j, :q+1],
             lw=lw, label=detector)
    #ax1.set_ylim(1e6, 1e11)
    ax4.set_xscale('linear')
    ax4.set_yscale('log')
    ax4.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax4.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax4.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax4.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax4.grid(True)
    h, l = ax4.get_legend_handles_labels()
    lgd1 = ax4.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.05),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax4.set_xlim(fd.t[off_set] , max(fd.t))
    ax4.set_ylim(2e1,1e5)
    # ----------------------------------------------------------------------------------------------------
    figure.savefig(PICS + '\\Frames\\frame_%d.jpeg' %i, quality=100)
    plt.close(figure)

600it [14:25,  1.44s/it]


In [20]:
images = []
filenames = np.array([
    PICS + 'Frames\\frame_%d.jpeg' %i
    for i in range(len(fd.t[off_set:]))
])
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(PICS + '%s_coarse.gif' %config['water']['model']['name'],
                images, fps=10)

In [21]:
vmax = 1.
vmin = 0.

In [22]:
# Random noise
noise = []
for i, _ in enumerate(fd.measured.keys()):
    noise.append(np.abs(np.random.normal(2e2, 1e2, len(fd.t))))
noise = np.array(noise)
x_grid = np.linspace(0., config['geometry']['volume']['x_length'], courseness)
y_grid = np.linspace(0., config['geometry']['volume']['y_length'], courseness)
X, Y = np.meshgrid(x_grid, y_grid)
all_points = []
for i in range(0, len(x_grid)):
    all_points.append(np.vstack((X[i], Y[i])).T)
for i, _ in tqdm(enumerate(fd.t[off_set:])):
    q = i + off_set
    figure, ((ax, ax3), (ax1, ax4)) = plt.subplots(2, 2, figsize=(std_size, std_size * 6. / 8.))
    # The water current
    x_vel = []
    y_vel = []
    grad = []
    for j in range(0, len(x_grid)):
        # x_vel_tmp, y_vel_tmp, vel_abs = fd._current.velocities(all_points[j], q + config['advanced']['starting step'])
        grad_tmp = fd._current.gradients(all_points[j], q + config['advanced']['starting step'])
        # x_vel.append(x_vel_tmp)
        # y_vel.append(y_vel_tmp)
        grad.append(grad_tmp)
    grad = np.array(grad)
    ax.imshow(grad.reshape(courseness, courseness), cmap='plasma',
              vmax=vmax, vmin=vmin, extent=[0., 26., 0., 10.])
    # ax.quiver(x_grid, y_grid, x_vel, y_vel, grad)
    # The emitters
    emission_mask = statistics[0][q].loc[:, 'is_emitting']
    ax.scatter(statistics[0][q].loc[~emission_mask, 'pos_x'].values,
               statistics[0][q].loc[~emission_mask, 'pos_y'].values,
               color='g',
               s=10.
    )
    ax.scatter(statistics[0][q].loc[emission_mask, 'pos_x'].values,
               statistics[0][q].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=10.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw/2, color='k')
    # Additional options
    ax.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax.title=ax.set_title('Population t = %.f s' %(fd.t[q]), fontsize=fontsize)
    ax.set_xlim(0., 10.)
    ax.set_ylim(3., 7.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, detector in enumerate(fd.measured.keys()):
        ax1.plot((fd.t)[:q+1],
             measured[0][detector].values[:q+1] + noise[j, :q+1],
             lw=lw, label=detector)
    #ax1.set_ylim(1e6, 1e11)
    ax1.set_xscale('linear')
    ax1.set_yscale('log')
    ax1.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax1.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax1.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax1.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax1.grid(True)
    h, l = ax1.get_legend_handles_labels()
    lgd1 = ax1.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.05),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax1.set_xlim(fd.t[off_set], max(fd.t))
    ax1.set_ylim(2e1,1e5)
    # ----------------------------------------------------------------------------------------------------
    # Water current
    ax3.imshow(grad.reshape(courseness, courseness), cmap='plasma',
               vmax=vmax, vmin=vmin, extent=[0., 26., 0., 10.])
    # Emitters
    emission_mask = statistics[1][q].loc[:, 'is_emitting']
    ax3.scatter(statistics[1][q].loc[~emission_mask, 'pos_x'].values,
               statistics[1][q].loc[~emission_mask, 'pos_y'].values,
               color='g',
               s=10.
    )
    ax3.scatter(statistics[1][q].loc[emission_mask, 'pos_x'].values,
               statistics[1][q].loc[emission_mask, 'pos_y'].values,
               color='r',
               s=10.
    )
    # Exclusion
    # theta goes from 0 to 2pi
    theta = np.linspace(0, 2*np.pi, 50)
    # the radius of the circle
    r = config['geometry']['exclusion']['radius']
    # compute x1 and x2
    x1 = r*np.cos(theta)
    x2 = r*np.sin(theta)
    ax3.plot(x1 + config['geometry']['exclusion']['x_pos'], x2 + config['geometry']['exclusion']['y_pos'],
            lw=lw/2, color='k')
    # Additional options
    ax3.set_xlabel(r'$X\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.set_ylabel(r'$Y\;[\mathrm{m}]$', fontsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax3.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    ax3.title=ax3.set_title('Population t = %.f s' %(fd.t[q]), fontsize=fontsize)
    ax3.set_xlim(0., 10.)
    ax3.set_ylim(3., 7.)
    # -----------------------------------------------------------------------------------------------------
    # The light emission curve
    for j, detector in enumerate(fd.measured.keys()):
        ax4.plot((fd.t)[:q+1],
             measured[1][detector].values[:q+1] + noise[j, :q+1],
             lw=lw, label=detector)
    #ax1.set_ylim(1e6, 1e11)
    ax4.set_xscale('linear')
    ax4.set_yscale('log')
    ax4.set_xlabel(r'$t\;[$s$]$', fontsize=fontsize)
    ax4.set_ylabel(r'$Unweighted\;Photons$', fontsize=fontsize)
    ax4.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
    ax4.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
    ax4.grid(True)
    h, l = ax4.get_legend_handles_labels()
    lgd1 = ax4.legend(h,l, loc=9, bbox_to_anchor=(0.9, +1.05),
                      ncol=1, fontsize=fontsize, handlelength=h_length,
                      fancybox=True, frameon=True)
    ax4.set_xlim(fd.t[off_set] , max(fd.t))
    ax4.set_ylim(2e1,1e5)
    # ----------------------------------------------------------------------------------------------------
    figure.savefig(PICS + '\\Frames\\frame_%d.jpeg' %i, quality=100)
    plt.close(figure)

600it [14:59,  1.50s/it]


In [23]:
images = []
filenames = np.array([
    PICS + 'Frames\\frame_%d.jpeg' %i
    for i in range(len(fd.t[off_set:]))
])
for filename in filenames[::3]:
    images.append(imageio.imread(filename))
imageio.mimsave(PICS + '%s_fine.gif' %config['water']['model']['name'],
                images, fps=10)